In [ ]:
!pip install opencv-python ultralytics

import cv2
import torch
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

model = YOLO('yolov8n.pt')


class ExplosionDetector:
    def __init__(self, model_path, confidence_threshold=0.5):
        self.model = YOLO(model_path)
        self.confidence_threshold = confidence_threshold
        self.warning_ranges = {
            'small': 50,
            'medium': 100,
            'large': 200
        }

    def detect_explosion(self, frame):
        results = self.model(frame)
        explosions = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                class_id = box.cls.item()
                confidence = box.conf.item()
                if confidence >= self.confidence_threshold:
                  if class_id == 0:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    explosion_size = (x2 - x1) * (y2 - y1)
                    explosions.append({'bbox': [x1, y1, x2, y2], 'size': explosion_size})

        return explosions

    def get_warning_level(self, explosion_size):
        if explosion_size < self.warning_ranges['small']:
            return 'small'
        elif explosion_size < self.warning_ranges['medium']:
            return 'medium'
        else:
            return 'large'
detector = ExplosionDetector('yolov8n.pt')

video_path = '/content/drive/MyDrive/Source/Explosion 2.mp4'
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    explosions = detector.detect_explosion(frame)
    for explosion in explosions:
        x1, y1, x2, y2 = explosion['bbox']
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        warning_level = detector.get_warning_level(explosion['size'])
        warning_range = detector.warning_ranges[warning_level]
        cv2.putText(frame, f"Explosion Detected! Warning Level: {warning_level}, Range:{warning_range} px", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# prompt: mounting google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
